In [ ]:
import os
import numpy as np
import argparse
import wandb

import torch
from csnet.train import train

In [ ]:
dir_train_gt = 'data/train/gt'
dir_train_img = 'data/train/img'
dir_val_gt = 'data/val/gt'
dir_val_img = 'data/val/img'

epochs = 20
batch_size = 2
lr = 0.0001
weight_decay = 0.0005
wce_loss_weight = 0.6
dice_loss_weight = 0.4

snapshot = 5
val_step = 5
model_path = 'model'

factor = 0.1
patience = 10

log_progress = True
wandb_project = 'CSNet Test'

In [ ]:
config = dict(
    epochs=epochs,
    batch_size=batch_size,
    lr=lr,
    weight_decay=weight_decay,
    wce_loss_weight=wce_loss_weight,
    dice_loss_weight=dice_loss_weight,
    factor=factor,
    patience=patience,
    snapshot=snapshot,
    val_step=val_step,
    model_path=model_path
)
    
config

In [ ]:
if log_progress:
    with open('/home/amedyukh/.wandb_api_key') as f:
        key = f.read()
    os.environ['WANDB_API_KEY'] = key
else:
    os.environ['WANDB_MODE'] = 'offline'

wandb.init(project=wandb_project, config=config)
config = argparse.Namespace(**config)

In [ ]:
train(dir_train_img, dir_train_gt, dir_val_img, dir_val_gt, config, log_progress)

In [ ]:
wandb.finish()